In [9]:
#  初始

import re
import os
import csv
from datetime import datetime

funs=[]
with open('E:/作业/杂物/web日志分析/archive/access.log', 'r', encoding='utf-8', errors='ignore') as f:
    lines = f.readlines()

def parse_log_line(line):
    log_pattern = re.compile(
        r'(?P<ip>\S+) - - \[(?P<time>.*?)\] "(?P<method>\S+) (?P<path>\S+) \S+" (?P<status>\d{3}) (?P<size>\d+|-) "(?P<referer>[^"]*)" "(?P<user_agent>[^"]*)"'
    )
    match = log_pattern.match(line.strip())  # 添加strip()去除换行符
    if match:
        return {
            'ip': match.group('ip'),           # 使用match对象的group方法
            'time': match.group('time'),
            'method': match.group('method'), 
            'path': match.group('path'),
            'status': match.group('status'),
            'size': match.group('size'),
            'referer': match.group('referer'),
            'user_agent': match.group('user_agent')
        }
    else:
        return None  # 如果匹配失败返回None

for i, line in enumerate(lines):
    parsed_line = parse_log_line(line)
    if parsed_line:                     # 只添加成功解析的行
        funs.append(parsed_line)

if funs:  # 确保有数据再写入
    with open("E:/作业/杂物/web日志分析/log01.csv", "w", newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=funs[0].keys())  # 变量名从data改为funs
        writer.writeheader()
        writer.writerows(funs)
    print(f"成功处理{len(funs)}行数据，已保存到CSV文件")
else:
    print("没有成功解析的数据")
print('运行完毕')

成功处理10364864行数据，已保存到CSV文件
运行完毕


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest

# 第1步：读取数据
print("正在读取日志数据...")
df = pd.read_csv("E:/作业/杂物/web日志分析/log01.csv")
print(f"总共有 {len(df)} 条日志记录")
print("数据的前5行：")
print(df.head())

# 第2步：提取简单特征（把文本转换成数字，机器学习只认识数字）
print("\n正在提取特征...")

def extract_simple_features(df):
    """提取简单的数字特征"""
    features = df.copy()
    
    # 长路径可能是攻击
    features['path_length'] = features['path'].str.len()
    
    # 是否有查询参数攻击常带参数
    features['has_query'] = features['path'].str.contains('\?').astype(int)
    
    # 是否是错误状态码
    features['status_code'] = features['status'].astype(int)
    features['is_error'] = (features['status_code'] >= 400).astype(int)
    
    # 请求大小把"-"替换成0
    features['size_num'] = features['size'].replace('-', '0').astype(int)
    
    # 可疑关键词
    suspicious_words = ['script', 'union', 'select', '..', 'admin', 'login']
    features['suspicious_count'] = 0
    for word in suspicious_words:
        features['suspicious_count'] += features['path'].str.contains(word, case=False).astype(int)
    
    # 特征6：User-Agent长度（太短或太长都可疑）
    features['ua_length'] = features['user_agent'].str.len()
    
    print("提取的特征：")
    print("- path_length: 路径长度")
    print("- has_query: 是否有查询参数")
    print("- is_error: 是否是错误状态码")
    print("- size_num: 响应大小")
    print("- suspicious_count: 可疑关键词数量")
    print("- ua_length: User-Agent长度")
    
    return features

# 调用特征提取函数
df_with_features = extract_simple_features(df)

# 选择用于检测的特征
feature_columns = ['path_length', 'has_query', 'is_error', 'size_num', 'suspicious_count', 'ua_length']
X = df_with_features[feature_columns].fillna(0)  # 把空值填成0

print(f"\n用于检测的特征矩阵大小：{X.shape}")
print("特征统计：")
print(X.describe())

# 第4步：使用孤立森林检测异常
print("\n开始检测异常...")

# 创建孤立森林模型（contamination=0.05表示假设5%的数据是异常）
isolation_forest = IsolationForest(
    contamination=0.05,  # 假设5%是异常
    random_state=42      # 固定随机种子，结果可重复
)

# 训练模型并预测（-1表示异常，1表示正常）
predictions = isolation_forest.fit_predict(X)

# 把预测结果加到原始数据中（转换为0正常，1异常）
df_with_features['is_anomaly'] = (predictions == -1).astype(int)

# 第5步：分析结果
print("\n=== 检测结果 ===")
normal_count = (df_with_features['is_anomaly'] == 0).sum()
anomaly_count = (df_with_features['is_anomaly'] == 1).sum()

print(f"正常请求：{normal_count} 条 ({normal_count/len(df)*100:.1f}%)")
print(f"异常请求：{anomaly_count} 条 ({anomaly_count/len(df)*100:.1f}%)")

# 第6步：查看异常样本
print("\n=== 异常样本分析 ===")
anomaly_data = df_with_features[df_with_features['is_anomaly'] == 1]

if len(anomaly_data) > 0:
    print("异常请求的IP地址TOP5：")
    print(anomaly_data['ip'].value_counts().head())
    
    print("\n异常请求的路径TOP5：")
    print(anomaly_data['path'].value_counts().head())
    
    print("\n异常请求的状态码分布：")
    print(anomaly_data['status'].value_counts())
    
    print("\n异常样本详情（前10条）：")
    columns_to_show = ['ip', 'method', 'path', 'status', 'path_length', 'suspicious_count']
    print(anomaly_data[columns_to_show].head(10))
    
    # 保存异常数据到文件
    anomaly_data.to_csv('异常请求.csv', index=False, encoding='utf-8')
    print(f"\n所有异常数据已保存到：异常请求.csv")
else:
    print("没有检测到异常请求")

# 第7步：简单的特征重要性分析
print("\n=== 特征分析 ===")
print("正常请求 vs 异常请求的特征对比：")

for feature in feature_columns:
    normal_avg = df_with_features[df_with_features['is_anomaly'] == 0][feature].mean()
    anomaly_avg = df_with_features[df_with_features['is_anomaly'] == 1][feature].mean()
    print(f"{feature}:")
    print(f"  正常请求平均值: {normal_avg:.2f}")
    print(f"  异常请求平均值: {anomaly_avg:.2f}")
    print(f"  差异倍数: {anomaly_avg/normal_avg if normal_avg != 0 else 'N/A':.2f}")
    print()

# 第8步：给出简单的判断依据
print("=== 异常判断依据 ===")
print("系统主要根据以下特征判断异常：")
print("1. 路径异常长 → 可能是注入攻击")
print("2. 包含可疑关键词 → 可能是恶意请求")  
print("3. 大量错误状态码 → 可能是扫描行为")
print("4. User-Agent异常 → 可能是自动化工具")
print("5. 组合特征异常 → 综合判断为可疑")

print("\n检测完成！请查看'异常请求.csv'文件了解详情。")

In [15]:
import pandas as pd

# 读取日志数据
print("读取日志...")
df = pd.read_csv("E:/作业/杂物/web日志分析/sampled_200k.csv")
print(f"总共有 {len(df)} 条记录")

# 创建一个空的列表来存放可疑的记录
suspicious_requests = []

print("\n开始检查每一条记录...")

# 逐行检查，用最简单的规则
for i, row in df.iterrows():
    ip = row['ip']
    path = row['path'] 
    status = row['status']
    user_agent = row['user_agent']
    
    # 给这条记录打分，分数越高越可疑
    suspicious_score = 0
    reasons = []  # 记录可疑的原因
    
    # 规则1：路径太长（超过100个字符）
    if len(path) > 100:
        suspicious_score += 2
        reasons.append("路径太长")
    
    # 规则2：包含危险词汇
    dangerous_words = ['script', 'union', 'select', 'drop', 'admin', '..', 'passwd']
    for word in dangerous_words:
        if word.lower() in path.lower():
            suspicious_score += 3
            reasons.append(f"包含危险词:{word}")
    
    # 规则3：错误状态码
    if str(status).startswith('4') or str(status).startswith('5'):
        suspicious_score += 1
        reasons.append("错误状态码")

    
    # 规则4：User-Agent太短（可能是工具）
    if len(user_agent) < 20:
        suspicious_score += 2
        reasons.append("User-Agent太短")
    
    # 规则5：包含特殊字符
    special_chars = ['<', '>', '%', '&', ';']
    for char in special_chars:
        if char in path:
            suspicious_score += 1
            reasons.append(f"包含特殊字符:{char}")
    
    # 如果分数超过3分，就认为可疑
    if suspicious_score >= 3:
        suspicious_requests.append({
            '序号': i+1,
            'IP地址': ip,
            '请求路径': path,
            '状态码': status,
            '可疑分数': suspicious_score,
            '可疑原因': '; '.join(reasons)
        })
    
    # 每处理1000条显示一次进度
    if (i + 1) % 1000 == 0:
        print(f"已处理 {i+1} 条记录...")

# 显示结果
print(f"\n=== 检查完成 ===")
print(f"总记录数: {len(df)}")
print(f"可疑记录数: {len(suspicious_requests)}")
print(f"可疑比例: {len(suspicious_requests)/len(df)*100:.1f}%")

# 如果有可疑记录，显示详情
if len(suspicious_requests) > 0:
    print(f"\n前10个最可疑的记录：")
    
    # 按可疑分数排序
    suspicious_requests.sort(key=lambda x: x['可疑分数'], reverse=True)
    
    for i, req in enumerate(suspicious_requests[:10]):
        print(f"\n第{i+1}个可疑记录：")
        print(f"  IP地址: {req['IP地址']}")
        print(f"  请求路径: {req['请求路径'][:100]}...")  # 只显示前100个字符
        print(f"  状态码: {req['状态码']}")
        print(f"  可疑分数: {req['可疑分数']}")
        print(f"  可疑原因: {req['可疑原因']}")
    
    # 保存到文件
    suspicious_df = pd.DataFrame(suspicious_requests)
    suspicious_df.to_csv('可疑请求列表.csv', index=False, encoding='utf-8')
    print(f"\n所有可疑记录已保存到: 可疑请求列表.csv")
    
    # 简单统计
    print(f"\n=== 简单统计 ===")
    
    # 统计可疑IP
    ip_counts = {}
    for req in suspicious_requests:
        ip = req['IP地址']
        ip_counts[ip] = ip_counts.get(ip, 0) + 1
    
    print("可疑IP排行榜:")
    sorted_ips = sorted(ip_counts.items(), key=lambda x: x[1], reverse=True)
    for i, (ip, count) in enumerate(sorted_ips[:5]):
        print(f"  第{i+1}名: {ip} (出现{count}次)")
    
    # 统计可疑原因
    reason_counts = {}
    for req in suspicious_requests:
        reasons = req['可疑原因'].split('; ')
        for reason in reasons:
            reason_counts[reason] = reason_counts.get(reason, 0) + 1
    
    print("\n最常见的可疑原因:")
    sorted_reasons = sorted(reason_counts.items(), key=lambda x: x[1], reverse=True)
    for i, (reason, count) in enumerate(sorted_reasons[:5]):
        print(f"  第{i+1}名: {reason} (出现{count}次)")

else:
    print("\n恭喜！没有发现明显可疑的记录。")

print(f"\n=== 分析完成 ===")
print("你可以打开 '可疑请求列表.csv' 文件查看详细结果")
print("\n这个程序的逻辑很简单:")
print("1. 逐行检查每个请求")
print("2. 根据简单规则给每个请求打分")
print("3. 分数高的就是可疑的")
print("4. 把可疑的保存下来让你查看")

读取日志...
总共有 200000 条记录

开始检查每一条记录...
已处理 1000 条记录...
已处理 2000 条记录...
已处理 3000 条记录...
已处理 4000 条记录...
已处理 5000 条记录...
已处理 6000 条记录...
已处理 7000 条记录...
已处理 8000 条记录...
已处理 9000 条记录...
已处理 10000 条记录...
已处理 11000 条记录...
已处理 12000 条记录...
已处理 13000 条记录...
已处理 14000 条记录...
已处理 15000 条记录...
已处理 16000 条记录...
已处理 17000 条记录...
已处理 18000 条记录...
已处理 19000 条记录...
已处理 20000 条记录...
已处理 21000 条记录...
已处理 22000 条记录...
已处理 23000 条记录...
已处理 24000 条记录...
已处理 25000 条记录...
已处理 26000 条记录...
已处理 27000 条记录...
已处理 28000 条记录...
已处理 29000 条记录...
已处理 30000 条记录...
已处理 31000 条记录...
已处理 32000 条记录...
已处理 33000 条记录...
已处理 34000 条记录...
已处理 35000 条记录...
已处理 36000 条记录...
已处理 37000 条记录...
已处理 38000 条记录...
已处理 39000 条记录...
已处理 40000 条记录...
已处理 41000 条记录...
已处理 42000 条记录...
已处理 43000 条记录...
已处理 44000 条记录...
已处理 45000 条记录...
已处理 46000 条记录...
已处理 47000 条记录...
已处理 48000 条记录...
已处理 49000 条记录...
已处理 50000 条记录...
已处理 51000 条记录...
已处理 52000 条记录...
已处理 53000 条记录...
已处理 54000 条记录...
已处理 55000 条记录...
已处理 56000 条记录...
已处理 57000 条记录...
已处理

In [12]:
import pandas as pd

# 读取原始 CSV（路径替换为你的实际路径）
df = pd.read_csv('E:/作业/杂物/web日志分析/log01.csv')  # 改成你的文件名！

# 抽取最多 20 万条（如果总数不足，就全保留）
sampled_df = df.sample(n=min(200000, len(df)), random_state=42)

# 保存成新的 CSV
sampled_df.to_csv('E:/作业/杂物/web日志分析/sampled_200k.csv', index=False)

print(f"原始数据共有 {len(df)} 条，已保存 {len(sampled_df)} 条到 sampled_200k.csv")


原始数据共有 10364864 条，已保存 200000 条到 sampled_200k.csv
